In [5]:
import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv1D
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras import preprocessing
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [6]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hendrik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
qa = pd.read_json("data/lemmatize_google_qa.json.xz")
jokes = pd.read_json("data/lemmatize_jokes.json.xz")
articles = pd.read_json("data/lemmatize_news.json.xz")

In [1]:
stop_words = stopwords.words('english')
stop_words.extend(["...", "'s", "wo", "n't", "'m", "ca", "'ll", "'re", "'ve", "'d", "ha", "´´", "´", "´´´", ",", "!", "'", ":", ";", '"', "\\", "``"])
stop_words.extend(["wa", "''", "the", "he", "my", "it"])

data_for_df = []

jokes_lemmas = jokes['lemmatize']

for i, joke in enumerate(jokes_lemmas):
    joke_words = [word for word in joke if word not in stop_words and word.isalpha()]
    joke_text = " ".join(joke_words)
    data_for_df.append({'text': joke_text, 'joke': 1, 'category': 0})
    
qa_lemmas = qa['lemmatize']

for i, q in enumerate(qa_lemmas):
    qa_words = [word for word in q if word not in stop_words and word.isalpha()]
    qa_text = " ".join(qa_words)
    data_for_df.append({'text': qa_text, 'joke': 0, 'category': 1})
    
articles_lemmas = articles['lemmatize']

for i, article in enumerate(articles_lemmas):
    article_words = [word for word in article if word not in stop_words and word.isalpha()]
    article_text = " ".join(article_words)
    article_text = re.sub("reporting .*", "", article_text)
    data_for_df.append({'text': article_text, 'joke': 0, 'category': 2})

NameError: name 'stopwords' is not defined

In [2]:
df = pd.DataFrame(data_for_df)
df.shape

NameError: name 'pd' is not defined

In [ ]:
df = df.drop_duplicates('text')
df.shape

In [21]:
df.head()
df = df[:1000]

,text,joke,category
0,seafood diet see food fish eat,1,0
1,shoe store al bundy fat woman came shoe store ...,1,0
2,stalin said dark humor like food everyone get,1,0
3,really hate dementia remember,1,0
4,hey guy got nice joke ya u,1,0


### Preparing data for Neural Network

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X, X_test, y, y_test = train_test_split(df['text'], df['joke'], test_size=0.05, stratify=df['joke'])

In [26]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [30]:
num_words = 15000
maxlen = 500

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_train)

tokenized_X_train = tokenizer.texts_to_sequences(X_train)
tokenized_X_val = tokenizer.texts_to_sequences(X_val)
tokenized_X_test = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(tokenized_X_train, maxlen=maxlen)
X_val_pad = pad_sequences(tokenized_X_val, maxlen=maxlen)
X_test_pad = pad_sequences(tokenized_X_test, maxlen=maxlen)

In [31]:
y_train_vec = to_categorical(y_train)
y_val_vec = to_categorical(y_val)
y_test_vec = to_categorical(y_test)

In [32]:
X_train_pad.shape

(1098513, 500)

In [33]:
X_test_pad.shape

(64241, 500)

In [34]:
X_val_pad.shape

(122057, 500)

### Neural Network training and testing

Model taken from the tensorflow guide: https://www.tensorflow.org/guide/keras/rnn. Plan was to add layers and tune it further but the results indicated this wasn't necessary and instead we put more work into preprocessing the data further and analysis.

In [35]:
model = Sequential()

model.add(Embedding(len(tokenizer.word_index), 32, input_length=maxlen))

model.add(Conv1D(256, 10, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 64)           960000    
_________________________________________________________________
lstm (LSTM)                  (None, 35)                14000     
_________________________________________________________________
dense (Dense)                (None, 2)                 72        
Total params: 974,072
Trainable params: 974,072
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.fit(X_train_pad, y_train_vec, validation_data=(X_val_pad, y_val_vec), epochs=3, verbose=1, batch_size=256)
model.evaluate(X_test_pad, y_test)

Train on 1098513 samples, validate on 122057 samples
Epoch 1/3
   3072/1098513 [..............................] - ETA: 5:37:01 - loss: 0.6603 - accuracy: 0.5558

KeyboardInterrupt: 

65.499